In [3]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
from datetime import datetime

# Lista das 5 ações brasileiras que você deseja monitorar
ACOES = ['VALE3.SA', 'PETR4.SA', 'ITUB4.SA', 'BBDC4.SA', 'BBAS3.SA']

#def coletar_dados_enriquecidos():
"""
Coleta o histórico e calcula features técnicas para um modelo de ML.
"""
periodo_dias = '90d'
data_hoje = datetime.now().strftime('%Y-%m-%d')
print(f"[{data_hoje}] - Iniciando a coleta de dados de {periodo_dias} para as ações: {ACOES}")


# Baixa os dados de todos os tickers com 'group_by=True' (padrão)
df_final = []
for acao in ACOES:
    df_temp = yf.download(tickers=acao, period=periodo_dias)
    tt1 = df_temp.reset_index(drop=False)
    tt1.columns = ['data','close','high','low','open','volume']
    tt1["ticket"] = acao
    df_final.append(tt1)

df = pd.concat(df_final)
print('Dataframe baixado')
### Processando


ultima_linhas = []
for acao in df["ticket"].unique():
    try:
        ticker_limpo = acao
        df_acao = df[df["ticket"] == acao]
        df_acao = df_acao.dropna()

        # Adiciona a coluna 'ticker' ao DataFrame da ação
        df_acao['ticker'] = ticker_limpo

        # --- Cálculo das Features Técnicas com pandas_ta ---
        df_acao['SMA_3d'] = ta.sma(df_acao['close'], length=3)
        df_acao['SMA_7d'] = ta.sma(df_acao['close'], length=7)
        df_acao['SMA_14d'] = ta.sma(df_acao['close'], length=14)

        df_acao['EMA_3d'] = ta.ema(df_acao['close'], length=3)
        df_acao['EMA_7d'] = ta.ema(df_acao['close'], length=7)
        df_acao['EMA_14d'] = ta.ema(df_acao['close'], length=14)

        df_acao['RSI_14d'] = ta.rsi(df_acao['close'], length=14)

        # Adiciona colunas diretamente ao DataFrame df_acao
        df_acao.ta.bbands(append=True)
        df_acao.ta.macd(append=True)

        # ['BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0']
        ['data', 'close', 'high', 'low', 'open', 'volume', 'ticket', 'ticker',
            'SMA_3d', 'SMA_7d', 'SMA_14d', 'EMA_3d', 'EMA_7d', 'EMA_14d', 'RSI_14d',
            'BBL_5_2.0_2.0', 'BBM_5_2.0_2.0', 'BBU_5_2.0_2.0', 'BBB_5_2.0_2.0',
            'BBP_5_2.0_2.0', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9']
        df_acao = df_acao.rename(columns={'BBL_5_2.0_2.0':'BBL_20','BBM_5_2.0_2.0':'BBM_20','BBU_5_2.0_2.0':'BBU_20'})
        # Renomeia as colunas de volume e fechamento

        # Seleciona a última linha de dados (mais recente)
        ultima_linha = df_acao.iloc[-1].to_frame().T

        colunas_modelo = [
            'data','ticker', 'close', 'volume', 'SMA_3d', 'SMA_7d', 'SMA_14d',
            'EMA_3d', 'EMA_7d', 'EMA_14d', 'RSI_14d', 'BBL_20',
            'BBM_20', 'BBU_20', 'MACD_12_26_9', 'MACDh_12_26_9',
            'MACDs_12_26_9'
        ]

        ultima_linha = ultima_linha[colunas_modelo]
        
        ultima_linhas.append(ultima_linha.values[0])
        print(f'Sucesso no ticket: {acao}')
    except:
        print(f'Erro no ticket: {acao}')
print('\n')
colunas = ultima_linha.columns
df_final = pd.DataFrame(ultima_linhas, columns=colunas)
display(df_final)




[2025-09-18] - Iniciando a coleta de dados de 90d para as ações: ['VALE3.SA', 'PETR4.SA', 'ITUB4.SA', 'BBDC4.SA', 'BBAS3.SA']


C:\Users\lpste\AppData\Local\Temp\ipykernel_7424\2632669858.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_temp = yf.download(tickers=acao, period=periodo_dias)
[*********************100%***********************]  1 of 1 completed
C:\Users\lpste\AppData\Local\Temp\ipykernel_7424\2632669858.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_temp = yf.download(tickers=acao, period=periodo_dias)
[*********************100%***********************]  1 of 1 completed
C:\Users\lpste\AppData\Local\Temp\ipykernel_7424\2632669858.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_temp = yf.download(tickers=acao, period=periodo_dias)
[*********************100%***********************]  1 of 1 completed
C:\Users\lpste\AppData\Local\Temp\ipykernel_7424\2632669858.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_temp = yf.download(tickers=acao

Dataframe baixado
Sucesso no ticket: VALE3.SA
Sucesso no ticket: PETR4.SA
Sucesso no ticket: ITUB4.SA
Sucesso no ticket: BBDC4.SA
Sucesso no ticket: BBAS3.SA




,data,ticker,close,volume,SMA_3d,SMA_7d,SMA_14d,EMA_3d,EMA_7d,EMA_14d,RSI_14d,BBL_20,BBM_20,BBU_20,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
0,2025-09-18,VALE3.SA,57.689999,17598700,57.730000,57.327143,56.597143,57.654255,57.308613,56.684047,76.069432,56.898563,57.538000,58.177437,1.020164,0.110425,0.909740
1,2025-09-18,PETR4.SA,31.410000,25584300,31.553333,31.455714,31.218571,31.494005,31.401212,31.212312,56.168671,31.066235,31.458000,31.849765,0.233778,0.070937,0.162841
2,2025-09-18,ITUB4.SA,38.490002,14950000,38.276667,38.005715,38.075714,38.321514,38.136233,37.965589,60.430610,37.136861,38.042001,38.947141,0.404392,-0.035429,0.439820
3,2025-09-18,BBDC4.SA,17.400000,25859200,17.323333,17.104286,16.975073,17.339951,17.178068,16.981707,63.823866,16.542686,17.166000,17.789313,0.309790,0.037653,0.272137
4,2025-09-18,BBAS3.SA,22.100000,35906800,21.970001,22.051429,21.433572,22.008670,21.874233,21.538209,61.047019,21.641332,22.018000,22.394669,0.491093,0.091593,0.399500


In [4]:
df_final

,data,ticker,close,volume,SMA_3d,SMA_7d,SMA_14d,EMA_3d,EMA_7d,EMA_14d,RSI_14d,BBL_20,BBM_20,BBU_20,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
0,2025-09-18,VALE3.SA,57.689999,17598700,57.730000,57.327143,56.597143,57.654255,57.308613,56.684047,76.069432,56.898563,57.538000,58.177437,1.020164,0.110425,0.909740
1,2025-09-18,PETR4.SA,31.410000,25584300,31.553333,31.455714,31.218571,31.494005,31.401212,31.212312,56.168671,31.066235,31.458000,31.849765,0.233778,0.070937,0.162841
2,2025-09-18,ITUB4.SA,38.490002,14950000,38.276667,38.005715,38.075714,38.321514,38.136233,37.965589,60.430610,37.136861,38.042001,38.947141,0.404392,-0.035429,0.439820
3,2025-09-18,BBDC4.SA,17.400000,25859200,17.323333,17.104286,16.975073,17.339951,17.178068,16.981707,63.823866,16.542686,17.166000,17.789313,0.309790,0.037653,0.272137
4,2025-09-18,BBAS3.SA,22.100000,35906800,21.970001,22.051429,21.433572,22.008670,21.874233,21.538209,61.047019,21.641332,22.018000,22.394669,0.491093,0.091593,0.399500
